<a href="https://colab.research.google.com/github/MadewChaves/Spark_Csv_to_Parquet/blob/main/File_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DATA TYPES / FILES - ESTUDOS

In [ ]:
# Instalação pyspark
! pip install pyspark

In [11]:
# Importação de bibliotecas
from google.colab import drive
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
# Montando driver
drive.mount('/content/drive')

In [ ]:
# Diretório apontando para local do arquivo
dir = '/content/drive/MyDrive/Colab Notebooks/Dados_gov'

In [14]:
# Criando a sessão spark
spark = SparkSession.builder.appName('Estudo_data_files').getOrCreate()

In [ ]:
# Criando o dataframe
file_name = '2024_Passagem'
df_passagem = spark.read.csv(f'{dir}/{file_name}.csv', header=True,sep=";", encoding='ISO-8859-1', inferSchema=True)

In [ ]:
# Tratamento para a padronização do nome de colunas do data frame
df_passagem_tratado = df_passagem.select([F.col(x).alias(x.replace('-','').replace(' ','_').lower()) for x in df_passagem.columns])

In [ ]:
# Tratamento das colunas valor_da_passagem e taxa_de_serviço que estavam com o tipo String e o separador ,
df_passagem_tratado = df_passagem_tratado.withColumn('valor_da_passagem',F.expr("cast(replace(valor_da_passagem,',','.') as float)")) \
                                          .withColumn('taxa_de_serviço',F.expr("cast(replace(valor_da_passagem,',','.') as float)"))

In [ ]:
# Gerando um novo arquivo Parquet a partir do base em CSV
df_passagem_tratado.write.mode('overwrite').parquet(f'{dir}/{file_name}.parquet')

In [84]:
# Coleta de tamanho dos dois arquivos, em CSV e PARQUET, por fim, a apresentação de ambos os tamanhos e a diferença em percentual entre eles
tamanho_csv = float(os.path.getsize(f"{dir}/{file_name}.csv"))
tamanho_parquet = float(os.path.getsize(f"{dir}/{file_name}.parquet"))
diferenca_percentual = round((tamanho_csv - tamanho_parquet) / tamanho_csv * 100, 2)

print("Tamanho do arquivo em CSV:",(os.path.getsize(f"{dir}/{file_name}.csv")),"bytes","\n\n","Tamanho do arquivo em PARQUET:",(os.path.getsize(f"{dir}/{file_name}.parquet")),"bytes","\n\n",f"Diferença entre os tamanhos em percentual: {diferenca_percentual} %")

Tamanho do arquivo em CSV: 37915234 bytes 

 Tamanho do arquivo em PARQUET: 4096 bytes 

 Diferença entre os tamanhos em percentual: 99.99 %
